## Naive Machine Translation
1. This is a Machine Traslation code.<br>
2. This code is a novice machine translation code which translates the <font color=#068DA9>English word to French word</font>.<br>
3. I used the concepts Transformation matrix. I optimised this transformation by reducing the loss function which is calculated using Forbinous Norm.<br>
4. I used <font color=#068DA9><b>K-Nearest neighnour</b></font> to find the closest similar french word.Similrity parameter used in this implemenatation is <b>cosine similarity</b>.<br> 
5. I used the subset of Google's word2VEC model for finding the word embeddings. 


In [1]:
import numpy as np
import pandas as pd
import nltk
import scipy
import pickle
import pdb  #python  debugger
import sklearn

import gensim
# Gensim is a Python library for topic modelling, document indexing and similarity retrieval with large corpora.
# Target audience is the natural language processing (NLP) and information retrieval (IR) community.
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords,twitter_samples
from os import getcwd
from gensim.models import keyedvectors
import  matplotlib.pyplot as plt

In [3]:
##The data
en_embedding_subset=pickle.load(open("en_embeddings.p","rb"))
fr_embedding_subset=pickle.load(open("fr_embeddings.p","rb"))


Looking at the data
en_embeddings_subset: the key is an English word, and the vaule is a
300 dimensional array, which is the embedding for that word.

'the': array([ 0.08007812,  0.10498047,  0.04980469,  0.0534668 , -0.06738281, ....
fr_embeddings_subset: the key is an French word, and the vaule is a 300
dimensional array, which is the embedding for that word.

'la': array([-6.18250e-03, -9.43867e-04, -8.82648e-03,  3.24623e-02,...

In [4]:
from utils import get_dict,consine_similarity

Hello
Hello world


In [5]:
en_fr_train=get_dict('en-fr.train.txt')
print('length of the training french dictionary is ',len(en_fr_train))
en_fr_test=get_dict('en-fr.test.txt')
print('length of the testing french dictionary is ',len(en_fr_test))

length of the training french dictionary is  5000
length of the testing french dictionary is  1500


In [6]:
def get_matrices(en_fr, english_vecs, french_vecs):
    x_l=list()
    y_l=list()
    #Creating sets for 
    fr_set=set(en_fr.values())
    english_word_set = english_vecs.keys()
    french_word_set=french_vecs.keys()
    for en_word in en_fr.keys():
        if (en_word in english_word_set and en_fr[en_word] in french_word_set):
            english_word_vec=english_vecs[en_word]
            fr_word=en_fr[en_word]
            french_word_vec = french_vecs[fr_word]
            x_l.append(english_word_vec)
            y_l.append(french_word_vec)
    X=np.vstack(x_l)
    Y=np.vstack(y_l)
    return X,Y

In [7]:
X_train,y_train=get_matrices(en_fr_train,en_embedding_subset,fr_embedding_subset)
X_test,y_test=get_matrices(en_fr_test,en_embedding_subset,fr_embedding_subset)
print("length of X_train is ",len(X_train))
print("length of y_train is ",len(y_train))

length of X_train is  4932
length of y_train is  4932


In [8]:
print(len(X_train[0]))

300


<h3>Transaltion part in Machine Translation</h3>
<font color=#068DA9><h3>Linear Transformation :-</h3></font>
<b><em>e*R=f </em><br>
now after getting f we look for nearest neighbours of f,the most nearest will be our transalation word</b>
<h5 style="color:#068DA9>">The loss function is =square(1/m||XR-Y|<sub>F</sub>|<sup>2</sup>)</h5>
where we are taking Forbinious Norm here 

In [9]:
from numpy import linalg as la
a=np.array([[1,2,3]
           ,[1,2,3]])
la.norm(a)

5.291502622129181

In [10]:
#Calulation of loss
def loss(X,Y,R):
    #here 
    # X =english word matrix
    # Y=French word matrix
    # R is Transformation matrix
    #value of ||XR-Y||
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector 
        space embeddings.
    Outputs:
        L: a matrix of dimension (m,n) - the value of the loss function for given X, Y and R.
    '''
    A=((np.dot(X,R))-Y)
    m=len(X)
    loss=((la.norm(A))**2)/m
    return loss

In [11]:
#Computation of Gradient Loss
def d_loss(X,Y,R):
    m=len(X)
    gredient_loss=2*(np.dot(X.transpose(),(np.dot(X,R)-Y)))/m
    return gredient_loss

In [12]:
def gradient_descent(X,Y,alpha=0.01,steps=100):
    X=np.array(X)
    Y=np.array(Y)
    R = np.random.rand(X.shape[1], X.shape[1])
    for i in range(0,steps):
        if i % 25 == 0:
            print(f"loss at iteration {i} is: {loss(X, Y, R):.4f}")
        R=R-(d_loss(X,Y,R)*alpha)
    return R

In [28]:
R_train=gradient_descent(X_train,y_train,alpha=0.8,steps=400)

loss at iteration 0 is: 967.5624
loss at iteration 25 is: 97.9774
loss at iteration 50 is: 26.8542
loss at iteration 75 is: 9.7979
loss at iteration 100 is: 4.3842
loss at iteration 125 is: 2.3338
loss at iteration 150 is: 1.4528
loss at iteration 175 is: 1.0376
loss at iteration 200 is: 0.8281
loss at iteration 225 is: 0.7167
loss at iteration 250 is: 0.6550
loss at iteration 275 is: 0.6197
loss at iteration 300 is: 0.5989
loss at iteration 325 is: 0.5864
loss at iteration 350 is: 0.5787
loss at iteration 375 is: 0.5738


In [14]:
def KNN(word_vector,candidates,K=1):
    similarity_l=list()
    for neighbor in candidates:
        cos=consine_similarity(word_vector, neighbor)
        similarity_l.append(cos)
    sorted_k=np.argsort(similarity_l)
    k_idx=sorted_k[-K:]
    return k_idx


In [24]:
v = np.array([1, 0, 1])
candidates = np.array(
    [[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[KNN(v, candidates, 1)])


[[2 0 1]]


In [25]:
def test_vocabulary(X,Y,R):
    F=np.dot(X,R)
    count=0
    for i in range(0,len(F)):
        predicted=KNN(F[i],Y,K=1)
        if predicted==i:
            count+=1
    
    accuracy=count/len(F)*100
    return accuracy


In [26]:
print(test_vocabulary(X_test,y_test,R_train))

55.21557719054242
